# Helpdesk Medium

In [1]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.`org.apache.spark::spark-sql:2.4.0`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app12-2")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", "hdfs://quickstart.cloudera:8020/user/hive/warehouse")
    .config("hive.metastore.uris", "thrift://quickstart.cloudera:9083")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "True")
    .getOrCreate()
}

import spark.implicits._

Loading spark-stubs, spark-hive
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.log4j.{Level, Logger}

import $ivy.$                                  

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@206ae3b8
import spark.implicits._

In [2]:
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

defined function sc
hiveCxt: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@7f39d2c0

In [3]:
// extend the DataFrame class to prettify the output of show()
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = -1, truncate: Int = 0) = {
        import xml.Utility.escape
        
        val data = if (limit < 0) ds.take(255) else ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

## 6.
List the Company name and the number of calls for those companies with more than 18 calls.

```
+------------------+----+
| Company_name     | cc |
+------------------+----+
| Gimmick Inc.     | 22 |
| Hamming Services | 19 |
| High and Co.     | 20 |
+------------------+----+
```

In [4]:
val issue = hiveCxt.table("sqlzoo.Issue")
val caller = hiveCxt.table("sqlzoo.Caller")
val customer = hiveCxt.table("sqlzoo.Customer")
(issue.join(caller, issue("Caller_id")===caller("Caller_id"))
 .join(customer, caller("Company_ref")===customer("Company_ref"))
 .select(customer("Company_ref"), customer("Company_name"), $"Call_ref")
 .groupBy("Company_ref", "Company_name")
 .agg(count("Call_ref").as("cc"))
 .filter($"cc">18)
 .drop("Company_ref")
 .orderBy("Company_name")
 .showHTML())

20/07/05 21:24:02 INFO metastore: Trying to connect to metastore with URI thrift://quickstart.cloudera:9083
20/07/05 21:24:02 INFO metastore: Connected to metastore.


Company_name,cc
Gimmick Inc.,22
Hamming Services,19
High and Co.,20


issue: DataFrame = [call_date: string, call_ref: int ... 5 more fields]
caller: DataFrame = [caller_id: int, company_ref: int ... 2 more fields]
customer: DataFrame = [company_ref: int, company_name: string ... 6 more fields]

## 7.
Find the callers who have never made a call. Show first name and last name

```
+------------+-----------+
| first_name | last_name |
+------------+-----------+
| David      | Jackson   |
| Ethan      | Phillips  |
+------------+-----------+
```

In [5]:
(caller.join(issue, caller("Caller_id")===issue("Caller_id"), "left")
 .select($"First_name", $"Last_name", caller("Caller_id"), $"Call_ref")
 .groupBy("Caller_id", "First_name", "Last_name")
 .agg(count("Call_ref").as("n")).orderBy("n")
 .filter($"n"<1)
 .drop("Caller_id", "n")
 .showHTML())

First_name,Last_name
Ethan,Phillips
David,Jackson


## 8.
For each customer show: Company name, contact name, number of calls where the number of calls is fewer than 5

```
+--------------------+------------+-----------+----+
| Company_name       | first_name | last_name | nc |
+--------------------+------------+-----------+----+
| Pitiable Shipping  | Ethan      | McConnell |  4 |
| Rajab Group        | Emily      | Cooper    |  4 |
| Somebody Logistics | Ethan      | Phillips  |  2 |
+--------------------+------------+-----------+----+
```

In [6]:
(issue.join(caller, issue("Caller_id")===caller("Caller_id"))
 .join(customer, caller("Company_ref")===customer("Company_ref"))
 .select(caller("Company_ref"), $"Company_name", $"Contact_id", $"Call_ref")
 .groupBy("Company_ref", "Company_name", "Contact_id")
 .agg(count("Call_ref").as("nc"))
 .filter($"nc"<5)
 .join(caller, $"Contact_id" === $"Caller_id", "inner")
 .select("Company_name", "First_name", "Last_name", "nc")
 .orderBy("Company_name")
 .showHTML())

Company_name,First_name,Last_name,nc
Pitiable Shipping,Ethan,McConnell,4
Rajab Group,Emily,Cooper,4
Somebody Logistics,Ethan,Phillips,2


## 9.
For each shift show the number of staff assigned. Beware that some roles may be NULL and that the same person might have been assigned to multiple roles (The roles are 'Manager', 'Operator', 'Engineer1', 'Engineer2').

```
+------------+------------+----+
| Shift_date | Shift_type | cw |
+------------+------------+----+
| 2017-08-12 | Early      |  4 |
| 2017-08-12 | Late       |  4 |
| 2017-08-13 | Early      |  3 |
| 2017-08-13 | Late       |  2 |
| 2017-08-14 | Early      |  4 |
| 2017-08-14 | Late       |  4 |
| 2017-08-15 | Early      |  4 |
| 2017-08-15 | Late       |  4 |
| 2017-08-16 | Early      |  4 |
| 2017-08-16 | Late       |  4 |
+------------+------------+----+
```

In [7]:
val shift = hiveCxt.table("sqlzoo.Shift")
val shift_det = (
    shift.selectExpr(
        "Shift_date", "Shift_type", 
        "stack(4, 'Manager', Manager, 'Operator', Operator, 'Engineer1', Engineer1, 'Engineer2', Engineer2)")
    .withColumnRenamed("col0", "role")
    .withColumnRenamed("col1", "person")
    .filter("person is not null")
    .dropDuplicates("Shift_date", "Shift_type", "person")
)
(shift_det.groupBy("Shift_date", "Shift_type")
 .agg(count("person").as("cw"))
 .orderBy("Shift_date", "Shift_type")
 .showHTML())

Shift_date,Shift_type,cw
2017-08-12,Early,4
2017-08-12,Late,4
2017-08-13,Early,3
2017-08-13,Late,2
2017-08-14,Early,4
2017-08-14,Late,4
2017-08-15,Early,4
2017-08-15,Late,4
2017-08-16,Early,4
2017-08-16,Late,4


shift: DataFrame = [shift_date: string, shift_type: string ... 4 more fields]
shift_det: Dataset[Row] = [Shift_date: string, Shift_type: string ... 2 more fields]

## 10.
Caller 'Harry' claims that the operator who took his most recent call was abusive and insulting. Find out who took the call (full name) and when.

```
+------------+-----------+---------------------+
| first_name | last_name | call_date           |
+------------+-----------+---------------------+
| Emily      | Best      | 2017-08-16 10:25:00 |
+------------+-----------+---------------------+
```

In [8]:
val staff = hiveCxt.table("sqlzoo.Staff")
(caller.filter($"First_name"==="Harry")
 .join(issue, Seq("Caller_id"))
 .join(staff, issue("Taken_by")===staff("Staff_code"))
 .select(staff("First_name"), staff("Last_name"), issue("Call_date"))
 .orderBy(desc("Call_date"))
 .limit(1)
 .showHTML())

First_name,Last_name,Call_date
Emily,Best,2017-08-16 10:25:00.0


staff: DataFrame = [staff_code: string, first_name: string ... 2 more fields]

In [9]:
spark.stop()